# Brownian Motion

In [ ]:
using KadanoffBaym
using LinearAlgebra

In [ ]:
using PyPlot

font_size = 16

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=font_size)
PyPlot.matplotlib.rc("axes", labelsize=font_size)
PyPlot.matplotlib.rc("xtick.major", size=8)
PyPlot.matplotlib.rc("ytick.major", size=8)
PyPlot.matplotlib.rc("xtick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("ytick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("xtick", top=true, direction="inout")
PyPlot.matplotlib.rc("ytick", right=true, direction="inout")

In [ ]:
function solution(N_0::Float64, D::Float64)
    F_ana(t1, t2) =(F[1, 1] - D/(2 * theta)) * exp(-theta * (t1 + t2)) + D/(2 * theta) * exp(-theta * abs(t1 - t2))

    f_vert(_, _, t1, t2) = [-theta * F[t1, t2]]
    f_diag(_, times, t)  = (println(times[t]); [-2theta * F[t, t] + D])

    F = GreenFunction(N_0 * ones(1,1), Symmetrical)
    
        sol = kbsolve(f_vert, f_diag, [F], (0.0, T), atol=1e-6, rtol=1e-4, dtini=0.5/2^8)
#     sol = kbsolve(f_vert, f_diag, [F], (0.0, T), atol=1e0, rtol=1e0, dtmax=0.5/2^8, dtini=0.5/2^8)

    return [sol.t, F.data, [F_ana(t1, t2) for t1 in sol.t, t2 in sol.t]]
end;

In [ ]:
T = 4.0/8
theta = 1.
t_scale = (iszero(theta) ? one(theta) : abs(theta))

N_0 = [1., 3., 5.]
Ds = [8., 4., 1.]

N_0 = [1.]
Ds = [8.]

s = mapreduce(solution, hcat, N_0, Ds);

In [ ]:
idx = 1
dts = map(x -> x[2] - x[1], zip(s[1, idx][1:end-1], s[1, idx][2:end]) |> collect);

In [ ]:
s[1, idx]

In [ ]:
diag(s[2, idx])[end]

In [ ]:
figure(figsize=(5, 3))
semilogy(s[1, idx][1:end-1], dts)

# xlim(0, theta * T)
# ylim(2e-4, 2e-2)
xlabel("\$\\theta t\$")
ylabel("\$\\mathrm{d} t\$")
tight_layout()

## Plotting

In [ ]:
cmap = "gist_heat";
colors = ["C0", "C1", "C2"];
lss = ["-", "--", "-."];

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
for k in eachindex(N_0)
    plot(s[1,k], diag(s[2,k]), label="\$xxx\$", lw=1.5, c=colors[k], ls=lss[k])
#     plot(s[1,k], s[3,k], ls="--", lw=4.0, alpha=0.4, c=colors[k])
end
xlim(0, theta * T)
ylim(0, 1.1 * Ds[1]/(2theta))
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\theta t\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot([], [], label="\$D/\\theta\$", c="w")
for k in eachindex(N_0)
    semilogy(s[1,k], abs.(diag(s[2,k])  .- diag(s[3,k])), lw=1.5, c=colors[k], label="\$"*string(Ds[k]/(theta))*"\$", ls=lss[k])
end

xlim(0, theta * T)
# ylim(1e-6, 1e-2)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\theta t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# # ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.25, w_pad=1, h_pad=0)

# savefig("brownian_motion_example_T.pdf")

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

In [ ]:
Y, X = meshgrid(s[1,1], s[1,1]);

In [ ]:
figure(figsize=(7, 3))

vmin = 0
vmax = Ds[1]/(2theta)
ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, s[2,1], cmap=cmap, rasterized=true, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\theta t\$")
ax.set_ylabel("\$\\theta t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, abs.(s[2,1] - s[3,1]) ./ abs.(s[3,1]), cmap="gist_gray_r", rasterized=true)#, vmin=0e-6, vmax=4e-6)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\theta t\$")
# ax.set_ylabel("\$\\theta t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_yticklabels([])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)
# savefig("brownian_motion_example_t_tp.pdf")#, dpi=256)

## Error scaling

In [ ]:
# epsilons = [2.0^(-k) for k in 6:19]
# err_data = [(0.0, 0.0, 0.0) for _ in 1:length(epsilons)]
# for (k, eps) in enumerate(epsilons)
#     print(k, ": ")
    
#     sol, _ = kbsolve(f_vert, f_diag, [F], tspan, dtini=1e-16, atol=eps*1e-2, rtol=eps)#, dtmax=eps)
# #     sol, _ = kbsolve(f_vert, f_diag, [F], tspan, dtini=1e-15, atol=1e-0, rtol=1e-0, dtmax=eps)
    
#     n = length(sol.t) - 1
#     print(n, ", ")
    
#     Y, X = np.meshgrid(sol.t[1:end], sol.t[1:end])
#     err_data[k] = (n, (1.0/n)^2 * norm(F_ana.(X, Y) - F.data, 1), eps)
# end

In [ ]:
# xdata = log10.([x[1] for x in err_data])
# ydata = log10.([x[2] for x in err_data]);

In [ ]:
# using LsqFit
# func_power = (n, p) -> 10^(p[2]) * n^(-p[1]);
# fit_func = (n, p) -> -p[1] .* n .+ p[2];
# fit_result = curve_fit(fit_func, xdata, ydata, [2.0, 1]);
# coef(fit_result)

In [ ]:
# figure(figsize=(7, 3))

# ax = subplot(121)
# plot(xdata, map(x -> fit_func(x, coef(fit_result)), xdata), "--k", lw=2,
#      label="\$\\mathcal{O}(h^{"*string(coef(fit_result)[1] |> x -> floor(x, sigdigits=4))*"})\$")
# plot(xdata, ydata, "s", lw=0, ms=5,
#      markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="k")
# xlim(0, 5)
# # ax.set_xticks([2.5, 2.75, 3.0, 3.25])
# xlabel("\$\\log(n)\$")
# ylabel("\$\\log(\\epsilon_{\\mathrm{abs}})\$")
# legend(loc="best", handlelength=1.8, frameon=false, borderpad=0, labelspacing=0)

# ax = subplot(122)
# plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "s", ms=5,
#      markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="k")
# ax.yaxis.set_label_position("right")
# # xlabel("\$\\log(\\mathrm{dtmax})\$")
# xlabel("\$\\log(\\mathrm{rtol})\$")
# ylabel("\$\\log(n)\$", labelpad=16)

# tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
# # savefig("error_scaling_brownian.pdf")

## Testing

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(sol.t, [F.data[k, k] for k in 1:n+1], "-", c="C0", label="\$xxx\$", lw=1.5)
plot(sol.t, F_ana.(sol.t, sol.t), c="C0", ls="--", lw=4.0, alpha=0.4)
xlim(0, lambda * T)
ylim(0, 1.1 * D/(2lambda))
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot(sol.t, abs.([F.data[k, k] for k in 1:n+1]  .- F_ana.(sol.t, sol.t)), ls="-", c="r", lw=1.5)
xlim(0, lambda * T)
ylim(0, )
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.1, w_pad=1, h_pad=0)

# savefig("brownian_motion_example.pdf")

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(sol.t, [F.data[1, k] for k in 1:n+1], "-", c="C0", label="\$xxx\$", lw=1.5)
plot(sol.t, F_ana.(1, sol.t), c="C0", ls="--", lw=3.0, alpha=0.5)
xlim(0, lambda * T)
ylim(0, D/(2lambda))
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda T\$")
ylabel("\$F(t, t)\$")

ax = subplot(122)
plot(sol.t, [F.data[k, k] for k in 1:n+1]  .- (F_ana.(sol.t, sol.t)), ls="-", c="r", lw=1.5)
xlim(0, lambda * T)
# ylim(0, 5.1)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\lambda T\$")
ylabel("\$F(t, t) - \\mathcal{F}(t, t)\$", labelpad=16)
ax.yaxis.set_label_position("right")
# ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.1, w_pad=1, h_pad=0)

# savefig("brownian_motion_example.pdf")

In [ ]:
figure(figsize=(7, 3))

vmin = 0
vmax = 1e-3
ax = plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, F_ana.(X, Y) - F.data, cmap=cmap, rasterized=true) # vmin=vmin, vmax=vmax
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
tight_layout()

In [ ]:
figure(figsize=(5, 3))

imshow(F.data |> imag, ColorMap(cmap))
colorbar(orientation="vertical")
ax = gca()
# ax.set_xticks([0, n/2 + 1, n + 1])
# ax.set_xticklabels([0, lambda .*  T/2, abs(lambda) .* T])
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
# ax.set_yticks([0, n/2 + 1, n + 1])
# ax.set_yticklabels([0, lambda .* T/2, abs(lambda) .*  T])
xlabel("\$\\lambda t'\$")
ylabel("\$\\lambda t\$")
# clim((0, 5))
tight_layout()